In [100]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb

import glob, io, json, os, re, requests, socket, subprocess
import pandas as pd

if not os.path.exists('python-utils'):
    subprocess.check_output('git clone https://github.com/CMU-CREATE-Lab/python-utils.git', shell=True)

def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']
    exec('\n'.join(src), globals())

exec_ipynb('python-utils/utils.ipynb')
notebook_wide_display()

In [101]:
# Read UPS names and serial numbers from this sheet
googleSheetUrl = 'https://docs.google.com/spreadsheets/d/1Klk0muDZlaXZkiyBQ8cIo9bMA6UESIPlLmFzHxszgl0/edit#gid=0'

In [102]:
Stat.set_service('UPS')

In [103]:
# Get serial numbers for APC devices
serialNumbers = []

for line in subprocess_check("lsusb -v -d 051d: | grep iSerial").split('\n'):
    tokens = line.split()
    if len(tokens) == 3:
        serialNumbers.append(tokens[2])

serialNumbers = sorted(serialNumbers)

print('Found %d APC UPSs connected by USB, serial numbers %s' % (len(serialNumbers), serialNumbers))

Found 2 APC UPSs connected by USB, serial numbers ['3B1923X20021', '3B1923X20093']


In [104]:
# Kill existing processes
subprocess_check('killall upsd usbhid-ups', ignore_error=True)

''

In [105]:
print('Creating /etc/nut/nut.conf')
open('/etc/nut/nut.conf', 'w').write('# Created by upsstat\nMODE=standalone\n')

Creating /etc/nut/nut.conf


37

In [106]:
print('Creating /etc/nut/ups.conf')
with open('/etc/nut/ups.conf', 'w') as ups_conf:
    ups_conf.write('# Created by upsstat\n')
    ups_conf.write('maxretry = 3\n')
    for i, serialNumber in enumerate(serialNumbers):
        print('Device %d: serial=%s' % (i, serialNumber))
        ups_conf.write('[%d]\n' % i)
        ups_conf.write('    driver = usbhid-ups\n')
        ups_conf.write('    serial = %s\n' % serialNumber)
        ups_conf.write('    port = auto\n')
    

Creating /etc/nut/ups.conf
Device 0: serial=3B1923X20021
Device 1: serial=3B1923X20093


In [107]:
for i, serialNumber in enumerate(serialNumbers):
    cmd = '/sbin/upsdrvctl start %d' % i
    subprocess_check(cmd, verbose=True)

upsdrvctl start 0
Using subdriver: APC HID 0.96
Network UPS Tools - Generic HID driver 0.41 (2.7.4)
USB communication driver 0.33
Network UPS Tools - UPS driver controller 2.7.4
upsdrvctl start 1
Using subdriver: APC HID 0.96
Network UPS Tools - Generic HID driver 0.41 (2.7.4)
USB communication driver 0.33
Network UPS Tools - UPS driver controller 2.7.4


In [108]:
subprocess_check('/sbin/upsd', verbose=True)

upsd
fopen /var/run/nut/upsd.pid: No such file or directory
listening on 127.0.0.1 port 3493
listening on ::1 port 3493
Connected to UPS [1]: usbhid-ups-1
Connected to UPS [0]: usbhid-ups-0


'fopen /var/run/nut/upsd.pid: No such file or directory\nlistening on 127.0.0.1 port 3493\nlistening on ::1 port 3493\nConnected to UPS [1]: usbhid-ups-1\nConnected to UPS [0]: usbhid-ups-0\n'

In [ ]:
def upsc(id):
    ret = {}
    for line in subprocess_check('upsc %s' % id).split('\n'):
        tokens = line.split(':')
        if len(tokens) == 2:
            key = tokens[0].strip()
            val = tokens[1].strip()
            try:
                val = float(val)
            except:
                pass
            ret[key] = val
    return ret


def checkUps(id, serialNumber, config):
    stat = upsc(id)
    errors = []
    maxLoad = 90 # percent
    if stat['input.voltage'] <= stat['input.transfer.low']:
        errors.append('Power out or input voltage low')
    if stat['ups.load'] > maxLoad:
        errors.append('High load %d%% (test >%d%%)' % (stat['ups.load'], maxLoad))

    name = serialNumber
    machines = ''
    
    details = 'In: %dV, Out load %d%%, Batt charge %d%%, S/N %s' % (
        stat['input.voltage'], stat['ups.load'], stat['battery.charge'], serialNumber)

    try:
        name = config.loc[serialNumber, 'Name'].strip()
        details += ', Powering %s' % config.loc[serialNumber, 'Machines'].strip()
        details += '.  <a href="%s">config</a>' % googleSheetUrl
    except:
        details +='.  Please put S/N in <a href="%s">Google sheet<a>' % googleSheetUrl

    if errors:
        Stat.down('; '.join(errors), details, host=name)
    else:
        Stat.up('Working normally', details, host=name)

def sheetUrl2CsvUrl(sheetUrl):
    tokens = sheetUrl.split('/')
    assert(len(tokens) == 7)
    assert(tokens[4] == 'd')
    docHash = tokens[5]
    assert(len(docHash) > 20)
    edit = tokens[6]
    assert edit[0:9] == 'edit#gid='
    gid = edit[9:]
    return 'https://docs.google.com/spreadsheets/d/' + docHash + '/export?format=csv&gid=' + gid

In [111]:
while True:
    sheetContent = requests.get(sheetUrl2CsvUrl(googleSheetUrl)).content
    config = pd.read_csv(io.BytesIO(sheetContent), na_filter=False, index_col='SerialNumber')

    for id, serialNumber in enumerate(serialNumbers):
        checkUps(id, serialNumber, config)
        
    sleep_until_next_period(60)

Stat.log up UPS NSH4629A Working normally In: 119V, Out load 13%, Batt charge 100%, S/N 3B1923X20021, Powering HAL40-41?.  <a href="https://docs.google.com/spreadsheets/d/1Klk0muDZlaXZkiyBQ8cIo9bMA6UESIPlLmFzHxszgl0/edit#gid=0">config</a>
Stat.log up UPS NSH4629B Working normally In: 119V, Out load 14%, Batt charge 100%, S/N 3B1923X20093, Powering HAL42-44?.  <a href="https://docs.google.com/spreadsheets/d/1Klk0muDZlaXZkiyBQ8cIo9bMA6UESIPlLmFzHxszgl0/edit#gid=0">config</a>
